In [1]:
import pandas as pd
import sqlite3

# ---------------------------------------------------------
# 1) DB1 öffnen und media_info laden
# ---------------------------------------------------------
conn_new = sqlite3.connect('archiv/filtered_tiktok_media.db')
df_media = pd.read_sql_query("SELECT * FROM media_info", conn_new)

print("Vor dem Merge (media_info):")
print(df_media[['id','url']].head(3))  # Beispiel-Ausgabe

# ---------------------------------------------------------
# 2) DB2 öffnen und media_metadata laden
#    Hier: id, url auslesen
# ---------------------------------------------------------
conn_meta = sqlite3.connect('archiv/tiktok_media_metadata.db')
df_meta = pd.read_sql_query("SELECT id, url FROM media_metadata", conn_meta)
conn_meta.close()

print("Daten aus media_metadata:")
print(df_meta[['id','url']].head(3))

# ---------------------------------------------------------
# 3) Per Merge abgleichen (left-join auf df_media)
#    suffixes: damit nicht beide "url"-Spalten kollidieren
# ---------------------------------------------------------
df_merged = pd.merge(
    df_media,    # linker DF (alle Zeilen behalten)
    df_meta,     # rechter DF
    on='id',
    how='left',
    suffixes=('', '_meta')  # dann wird df_meta.url => 'url_meta'
)

# ---------------------------------------------------------
# 4) Die url-Spalte aus df_meta in df_media übertragen
#    (falls in df_meta.url Werte vorhanden sind)
# ---------------------------------------------------------
df_merged['url'] = df_merged['url_meta']

# Nicht mehr benötigte Spalte entfernen
df_merged.drop(columns=['url_meta'], inplace=True)

# Ausgabe zum Testen
print("Nach dem Merge:")
print(df_merged[['id','url']].head(5))

# ---------------------------------------------------------
# 5) Ergebnis zurück in 'filtered_tiktok_media.db' speichern
#    ACHTUNG: 'replace' überschreibt die Tabelle komplett.
# ---------------------------------------------------------
df_merged.to_sql("media_info", conn_new, if_exists="replace", index=False)

conn_new.close()

print("Fertig! 'media_info' wurde mit den URL-Werten aktualisiert.")


Vor dem Merge (media_info):
                    id   url
0  7453041971486133536  None
1  7446314527685201185  None
2  7430740655854390560  None
Daten aus media_metadata:
                    id                                                url
0  7453041971486133536  https://www.tiktok.com/@gereonbollmann/video/7...
1  7446314527685201185  https://www.tiktok.com/@gereonbollmann/video/7...
2  7430740655854390560  https://www.tiktok.com/@gereonbollmann/video/7...
Nach dem Merge:
                    id                                                url
0  7453041971486133536  https://www.tiktok.com/@gereonbollmann/video/7...
1  7446314527685201185  https://www.tiktok.com/@gereonbollmann/video/7...
2  7430740655854390560  https://www.tiktok.com/@gereonbollmann/video/7...
3  7429638543347076385  https://www.tiktok.com/@gereonbollmann/video/7...
4  7424819005254405408  https://www.tiktok.com/@gereonbollmann/video/7...
Fertig! 'media_info' wurde mit den URL-Werten aktualisiert.
